<a href="https://colab.research.google.com/github/tm70/friendly-couscous/blob/master/LFW_Conv_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

In [2]:
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]
# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

# Split into a training set and a test set using a stratified k fold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_test.shape)

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
X_train = X_train.reshape(len(X_train), h, w, 1)
X_test = X_test.reshape(len(X_test), h, w, 1)

print(X_train.shape)
print(X_test.shape)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7
(966, 1850)
(322, 1850)
(966, 50, 37, 1)
(322, 50, 37, 1)


In [3]:
filters = 32
size = (3,3)
batch_size = 256
epochs = 50

#create the network
inputs = Input(shape=(h,w,1))
net1 = Conv2D(filters, size, padding="same", activation="relu")(inputs)
pool1 = MaxPooling2D(pool_size=(2,2))(net1)
net2 = Conv2D(filters, size, padding="same", activation="relu")(pool1)
pool2 = MaxPooling2D(pool_size=(2,2))(net2)
flat = Flatten()(pool2)
flat = Flatten()(inputs)
dense1 = Dense(128, activation="relu")(flat)
output = Dense(n_classes, activation="softmax")(dense1)

model = Model(inputs, output)

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_test, y_test))

Epoch 1/50
4/4 [==============================] - 0s 46ms/step - loss: 2.3201 - accuracy: 0.2050 - val_loss: 1.9256 - val_accuracy: 0.2267
Epoch 2/50
4/4 [==============================] - 0s 12ms/step - loss: 1.9512 - accuracy: 0.3447 - val_loss: 1.7481 - val_accuracy: 0.3634
Epoch 3/50
4/4 [==============================] - 0s 12ms/step - loss: 1.7692 - accuracy: 0.3230 - val_loss: 1.6262 - val_accuracy: 0.4534
Epoch 4/50
4/4 [==============================] - 0s 12ms/step - loss: 1.7742 - accuracy: 0.3975 - val_loss: 1.6210 - val_accuracy: 0.4534
Epoch 5/50
4/4 [==============================] - 0s 13ms/step - loss: 1.6733 - accuracy: 0.4151 - val_loss: 1.5644 - val_accuracy: 0.4845
Epoch 6/50
4/4 [==============================] - 0s 12ms/step - loss: 1.6373 - accuracy: 0.4172 - val_loss: 1.5109 - val_accuracy: 0.4596
Epoch 7/50
4/4 [==============================] - 0s 12ms/step - loss: 1.5885 - accuracy: 0.4337 - val_loss: 1.5141 - val_accuracy: 0.5000
Epoch 8/50
4/4 [===========

In [5]:
predictions = model.predict(X_test)

In [6]:
#reverse one-hot encoding
predictions = np.argmax(predictions, axis=1)

#print performance to compare with previous RF versions
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      0.54      0.67        13
           1       0.84      0.82      0.83        60
           2       0.64      0.52      0.57        27
           3       0.73      0.93      0.82       146
           4       0.85      0.44      0.58        25
           5       1.00      0.27      0.42        15
           6       0.60      0.50      0.55        36

    accuracy                           0.74       322
   macro avg       0.79      0.57      0.63       322
weighted avg       0.76      0.74      0.73       322

